In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# compute base forecast no coherent
from statsforecast.models import AutoARIMA, AutoETS, AutoCES, AutoTheta, HoltWinters, CrostonOptimized, SeasonalNaive
from statsforecast.core import StatsForecast

#obtain hierarchical reconciliation methods and evaluation
from hierarchicalforecast.methods import BottomUp, MinTrace, TopDown, OptimalCombination
from hierarchicalforecast.utils import aggregate, HierarchicalPlot
from hierarchicalforecast.core import HierarchicalReconciliation
from hierarchicalforecast.evaluation import HierarchicalEvaluation

In [6]:
df = pd.read_excel('./HCP_Data_KDAG_Hackathon.xlsx', parse_dates=['Time_Period'])
df.head()

,Physician_ID,Time_Period,Brand_Rx,Market_Rx,Sales_Rep_Calls,Samples_Dropped,Physician_Segment,Emails_Delivered,Speaker_Programs_Attended,Vouchers_Dropped,Specialty
0,axt00001,2019-01-04,0.0,2.80567,1,0,3-Low,0,0,0,Dermatologist
1,axt00001,2019-01-11,0.0,20.57312,1,0,3-Low,0,0,0,Dermatologist
2,axt00001,2019-01-18,0.0,6.16010,1,0,3-Low,0,0,0,Dermatologist
3,axt00001,2019-01-25,0.0,8.95501,1,5,3-Low,0,0,0,Dermatologist
4,axt00001,2019-02-01,0.0,9.13793,1,0,3-Low,0,0,0,Dermatologist


In [7]:
df = df.drop(['Speaker_Programs_Attended', 'Vouchers_Dropped', 'Market_Rx', 'Sales_Rep_Calls', 'Samples_Dropped', 'Emails_Delivered'], axis=1)
df = df[['Physician_Segment', 'Specialty', 'Physician_ID', 'Time_Period', 'Brand_Rx']]
df['Specialty'] = df['Specialty'].map({'Dermatologist':'D', 'General Physician':'GP', 'Nurse Practitioner':'NP'})
df['Physician_Segment'] = df['Physician_Segment'].map({'3-Low':'L', '2-Medium':'M', '1-High':'H'})
df = df.rename({'Time_Period': 'ds', 'Brand_Rx': 'y'}, axis=1)
df.insert(0, 'Job', 'HCP')
df['y'] = df['y'] + 1e-7 # to avoid log(0)
df.head()

,Job,Physician_Segment,Specialty,Physician_ID,ds,y
0,HCP,L,D,axt00001,2019-01-04,1.000000e-07
1,HCP,L,D,axt00001,2019-01-11,1.000000e-07
2,HCP,L,D,axt00001,2019-01-18,1.000000e-07
3,HCP,L,D,axt00001,2019-01-25,1.000000e-07
4,HCP,L,D,axt00001,2019-02-01,1.000000e-07


In [8]:
spec = [
    ['Job'],
    ['Job', 'Physician_Segment'], 
    ['Job', 'Specialty'], 
    ['Job', 'Physician_ID'],
    ['Job', 'Physician_Segment', 'Specialty'], 
    ['Job', 'Physician_Segment', 'Physician_ID'], 
    ['Job', 'Specialty', 'Physician_ID'],
    ['Job', 'Physician_Segment', 'Specialty', 'Physician_ID']
]

In [9]:
Y_df, S_df, tags = aggregate(df=df, spec=spec)
Y_df = Y_df.reset_index()

In [10]:
base_ts = tags['Job/Physician_Segment/Specialty/Physician_ID']

In [11]:
fcst = StatsForecast(df=Y_df,
                     models=[AutoARIMA(season_length=7),
                            AutoCES(season_length=7),
                            AutoTheta(season_length=7),
                            HoltWinters(season_length=7),
                            CrostonOptimized(),
                            ], 
                     freq='W-FRI', n_jobs=-1, fallback_model = SeasonalNaive(season_length=7))
Y_hat_df = fcst.forecast(h=1, fitted=True, level=[90])
Y_fitted_df = fcst.forecast_fitted_values()

/opt/miniconda3/envs/ml_env/lib/python3.8/site-packages/statsmodels/tsa/stattools.py:681: RuntimeWarning: invalid value encountered in true_divide
  acf = avf[: nlags + 1] / avf[0]
/opt/miniconda3/envs/ml_env/lib/python3.8/site-packages/statsmodels/tsa/stattools.py:681: RuntimeWarning: invalid value encountered in true_divide
  acf = avf[: nlags + 1] / avf[0]
/opt/miniconda3/envs/ml_env/lib/python3.8/site-packages/statsforecast/arima.py:864: RuntimeWarning: divide by zero encountered in log
  return 0.5 * np.log(res)
/opt/miniconda3/envs/ml_env/lib/python3.8/site-packages/statsforecast/arima.py:864: RuntimeWarning: divide by zero encountered in log
  return 0.5 * np.log(res)
/opt/miniconda3/envs/ml_env/lib/python3.8/site-packages/statsforecast/arima.py:864: RuntimeWarning: divide by zero encountered in log
  return 0.5 * np.log(res)
/opt/miniconda3/envs/ml_env/lib/python3.8/site-packages/statsforecast/arima.py:864: RuntimeWarning: divide by zero encountered in log
  return 0.5 * np.log

In [ ]:
Y_hat_df.to_csv('Y_hat_df.csv', index=False)
Y_fitted_df.to_csv('Y_fitted_df.csv', index=False)

In [12]:
reconcilers = [
    TopDown(method='proportion_averages'),
    MinTrace(method='mint_shrink'),
    MinTrace(method='ols'),
    OptimalCombination(method='ols'),
]
hrec = HierarchicalReconciliation(reconcilers=reconcilers)
Y_rec_df = hrec.reconcile(Y_hat_df=Y_hat_df, Y_df=Y_fitted_df, 
                          S=S_df, tags=tags, level=[90])

: 

: 

In [ ]:
Y_rec_df.to_csv('Y_rec_df.csv', index=False)

In [ ]:
Y_out = Y_rec_df[Y_rec_df['unique_id'].isin(base_ts)]